In [3]:
import os
import cv2
import numpy as np
import random
class MRIDataRead():
    def __init__(self, root,newdim,batch_size,channel_no):
        if root == '':
            root = "D://datasets//MRI_Mahdieh_Datasets//task1//"
        self.root = root
        self.dirs = os.listdir(root)
        self.newdim = newdim
        self.channel_num = 1
        self.channel_no = channel_no
        self.batch_size = batch_size
    def ReadData(self):   
        mri_list = []
        target_list = []
        patient_list = []
        patients_compact_list = []
        targets_compact_list = []
        root = self.root
        p_index = -1
        for dir1 in self.dirs:
            path = root + dir1 + "//"
            dirs2 = os.listdir(path)
            target1 = dir1
            
            for dir2 in dirs2:
                p_index = p_index + 1
                path2 = path + dir2 + "//"
                files = os.listdir(path2)
                patient_mri = []
                patients_compact_list.append(p_index)
                targets_compact_list.append(dir1)
                for file1 in files:
                    img = cv2.imread(path2+file1,cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img,(self.newdim,self.newdim))
                    try:
                        self.channel_num = img.shape[2]
                    except:
                        self.channel_num = 1
                    img = img.reshape(-1)
                    patient_mri.append(img)
                data_batch, target_batch = self.MRIBatch(patient_mri,target1)
                patient_index_tmp = np.ones((len(target_batch),1))
                patient_index_tmp = patient_index_tmp.astype(int)
                patient_index_tmp = patient_index_tmp * p_index
                mri_list.append(data_batch)
                target_list.append(target_batch)
                patient_list.append(patient_index_tmp)
        return mri_list,target_list,patient_list,patients_compact_list,targets_compact_list,self.channel_num
    def SelectIndices(self,imageNo):
        indices = []
        bs = round(imageNo/self.channel_no)
        for k in range(self.channel_no):
            random_integer = random.randint((k*bs+1), (k+1)*bs)
            if random_integer >= imageNo:
                random_integer = imageNo -1
            indices.append(random_integer)
        indices = np.array(indices)
        return indices
        
    def MRIBatch(self,patient_mri,target):
        data_batch = []
        target_batch = []
        targets = np.array(target)
        patient_mri = np.array(patient_mri)
        target1 = 0
        if target == 'grade1':
            target1 = 1
        if target == 'grade2':
            target1 = 2
        if target == 'grade3':
            target1 = 3
        if target == 'grade4':
            target1 = 4
        for k in range(self.batch_size):
            tmp_list = []
            indices = self.SelectIndices(patient_mri.shape[0])
            for i in indices:
                tmp_list.append(patient_mri[i])
            data_batch.append(tmp_list)
            target_batch.append(target1)
        return data_batch, target_batch


In [4]:
root = "D://datasets//MRI_Mahdieh_Datasets//task1//"
newdim = 128
batch_size = 20
channel_no = 3
mri_data_read = MRIDataRead(root,newdim,batch_size,channel_no)
mri_list,target_list,patient_index,patients_compact_list,targets_compact_list,channel_num = mri_data_read.ReadData()
target_array = np.array(target_list)
print(target_array.shape)
patient_array = np.array(patient_index)
print(patient_array.shape)
mri_array = np.array(mri_list)
print(mri_array.shape)
patients_compact_array = np.array(patients_compact_list)
targets_compact_array = np.array(targets_compact_list)
from sklearn.model_selection import train_test_split
train_patients, test_patients, _, _ = train_test_split(patients_compact_list, targets_compact_list,test_size=0.2)


(88, 20)
(88, 20, 1)
(88, 20, 3, 16384)


In [5]:
train_data = mri_array[train_patients,:,:,:]
train_labels = target_array[train_patients,:]
test_data = mri_array[test_patients,:,:,:]
test_labels = target_array[test_patients,:]
train_data = train_data.reshape(train_data.shape[0]*train_data.shape[1],newdim,newdim,-1)
train_labels = train_labels.reshape(train_labels.shape[0]*train_labels.shape[1],1)
test_data = test_data.reshape(test_data.shape[0]*test_data.shape[1],newdim,newdim,-1)
test_labels = test_labels.reshape(test_labels.shape[0]*test_labels.shape[1],1)


In [6]:
print(train_data.shape,train_labels.shape)

(1400, 128, 128, 3) (1400, 1)
